In [1]:
import os
import pandas as pd
from decimal import Decimal
import numpy as np
from datetime import datetime, timedelta
from dotenv import load_dotenv
import yfinance as yf


import matplotlib.pyplot as plt
import plotly.express as px
import pytz  # Make sure to import pytz for timezone handling
import seaborn as sns


import requests
import csv
import json

import warnings



from sklearn.linear_model import LinearRegression

In [2]:
load_dotenv()

API_KEY = os.getenv("alpha_vantage_api_key")

In [3]:
# Addtional setting session
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")
pd.options.mode.copy_on_write = True

In [77]:
# Parameters section

alpha_vantage_api_key = API_KEY # FREE TIER API rate limit is 25 requests per day
alpha_vantage_function = {
    'core':[
        'TIME_SERIES_INTRADA'
        ,'TIME_SERIES_DAILY' # this is daily time series quote
        ,'TIME_SERIES_DAILY_ADJUSTED' # this is daily time series adjusted by split/dividend-adjusted
        ,'GLOBAL_QUOTE'
    ]
    ,'fundmental':[
    'INCOME_STATEMENT'
    ,'BALANCE_SHEET' # this is daily time series quote
    ,'CASH_FLOW' # this is daily time series adjusted by split/dividend-adjusted
    ,'EARNINGS'
    ,'EARNINGS_CALENDAR'
]
}

# Custmized
ticker_symbols = [

    'GOOG'
    # ,'MSFT'
    ,'KR'
    # ,'AS'
    # ,'CE'
    # ,'APTV'
    # ,'MSFT'
    # ,'NVDA'
    # ,'CEPU'
    # ,'DHI'
    # ,'TSM'
    # ,'ULTA'
]


# # Consumer Staples
# ticker_symbols = [
#     'MO',
#     'ADM',
#     # 'BF.B', # DATA ISSUE
#     'BG',
#     'CPB',
#     'CHD',
#     'CLX',
#     'KO',
#     'CL',
#     'CAG',
#     'STZ',
#     'COST',
#     'DG',
#     'DLTR',
#     'EL',
#     'GIS',
#     'HSY',
#     'HRL',
#     'K',
#     'KVUE',
#     'KDP',
#     'KMB',
#     'KHC',
#     'KR', # DATA ISSUE
#     'LW',
#     'MKC', # DATA ISSUE
#     'TAP',
#     'MDLZ',
#     'MNST',
#     'PEP',
#     'PM',
#     'PG',
#     'SJM',
#     'SYY',
#     'TGT',
#     'TSN',
#     'WBA',
#     'WMT'
# ]

# # Energy
# ticker_symbols = [
# 'APA',
#  'BKR', 
#  'CVX',
#  'COP',
#  'CTRA',
#  'DVN',
#  'FANG',
#  'EOG',
#  'EQT',
#  'XOM',
#  'HAL', 
#  'HES', 
#  'KMI',
#  'MRO',
#  'MPC',
#  'OXY',
#  'OKE', 
#  'PSX',
#  'SLB', 
#  'TRGP',
#  'VLO',
#  'WMB'
#  ]


# Consumer Discretionary
# ticker_symbols = [
#  'ABNB',
#  'AMZN',
#  'APTV',
#  'AZO',
#  'BBWI',
#  'BBY',
#  'BKNG',
#  'BWA',
#  'CZR',
#  'KMX',
#  'CCL',
#  'CMG',
#  'DRI',
#  'DECK',
#  'DPZ',
#  'DHI',
#  'EBAY',
#  'ETSY',
#  'EXPE',
#  'F',
#  'GRMN',
#  'GM',
#  'GPC',
#  'HAS',
#  'HLT',
#  'HD',
#  'LVS',
#  'LEN',
#  'LKQ',
#  'LOW',
#  'LULU',
#  'MAR',
#  'MCD',
#  'MGM',
#  'MHK',
#  'NKE',
#  'NCLH',
#  'NVR',
#  'ORLY',
#  'POOL',
#  'PHM',
#  'RL',
#  'ROST',
#  'RCL',
#  'SBUX',
#  'TPR',
#  'TSLA',
#  'TJX',
#  'TSCO',
#  'ULTA',
#  'WYNN',
#  'YUM'
#  ]


# # # Basic Materials
# ticker_symbols = [
#     'LIN'
#     ,'SHW'
#     ,'APD'
#     ,'FCX'
#     ,'ECL'
#     ,'CTVA'
#     ,'NEM'
#     ,'VMC'
#     ,'DD'
#     ,'MLM'
#     ,'NUE'
#     ,'DOW'
#     ,'PPG'
#     # ,'SW'
#     ,'IFF'
#     ,'LYB'
#     ,'PKG'
#     ,'IP'
#     ,'STLD'
#     ,'BALL'
#     ,'AVY'
#     ,'CF'
#     ,'AMCR'
#     ,'EMN'
#     ,'ALB'
#     ,'CE'
#     ,'MOS'
#     ,'FMC'
# ]


# Time intelligent parameters
window_days = 90
end_date = datetime.now()
start_date = end_date - timedelta(days=window_days)
earning_calendar = [
    3  # this will return next 1 qtr forecast earning; nowadays the earning calendar only shows the next 1 qtr forecast earning
    ,6  # this will return next 2 qtr forecast earning
    ,12  # this will return next 4 qtr forecast earning
]

PE_yr_range = 6 # this will return x-1 yr PE range

ticker_dict_json = {}
ticker_dict_pd = {}

In [78]:
def convert_to_numeric_or_zero(value):
    try:
        # Try to convert the string to a float (or int)
        num = round(float(value), 2)
        return num  # Return the original value if it can be converted to a number
    except ValueError:
        # If conversion fails, return '0'
        return 0

# PE TTM Valuation
It is a stock screener partitioned by ticker level

In [79]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df = pd.DataFrame()
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])


    # MA200 calculation
    Daily_stock_df['MA200'] = Daily_stock_df.sort_index(ascending=True)['stock_price'].rolling(window=200).mean()

    # Calculate the slope of the MA200 using linear regression
    def calculate_slope(series):
        # Create a time index (0, 1, 2, ..., n) for the linear regression
        x = np.arange(len(series)).reshape(-1, 1)
        y = series.values.reshape(-1, 1)

        # Fit the linear regression model
        model = LinearRegression()
        model.fit(x, y)

        # Return the slope
        return model.coef_[0][0]


    # Apply the slope calculation to the MA200 values
    Daily_stock_df['MA200_slope'] = Daily_stock_df.sort_index(ascending=True)['MA200'].rolling(window=20).apply(calculate_slope, raw=False)



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)

    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])



    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - 6) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate metrics
            if Monthly_stock_df.shape[0] <= annualEPS_df.shape[0]: # if Monthly_stock_df has less records than annualEPS_df, choose the mini length record
                annualEPS_df = annualEPS_df[:(Monthly_stock_df.shape[0])]

                annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values
                annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df["PE"].mean().round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_std"] = np.std(annualEPS_df["PE"]).round(2)
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] + annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围上限
                annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = (annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"] - annualEPS_df[f"PE_{PE_yr_range-1}yr_std"]).round(2) # 这个是PE的波动范围下限



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue


    # # forecast 1 qtr earnings from alpha vantage API
    # for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
    CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol={symbol}&horizon=12month&apikey={alpha_vantage_api_key}'
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)

        forecast_earanings_df = pd.DataFrame(
            columns=my_list[0]
            ,data=my_list[1::]
            )
        
        if forecast_earanings_df['estimate'].head(1).values != '':
            latest_projected_EPS = float(forecast_earanings_df['estimate'].head(1).values)
        else:
            latest_projected_EPS = 0


    # forecast 1 year earnings from yf API, forwardPE, PEG
    # yf data
    yf_data = yf.Ticker(symbol).info
    # alpha vintage data
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()
    
    for key, value in data.items():
        if key == 'PEGRatio':
            PEG_12month_projected = convert_to_numeric_or_zero(value=value)


        if key == 'ForwardPE':
           PE_12month_projected = convert_to_numeric_or_zero(value=value)


    forecast_earnings_keys = [
        'forwardEps'
    ]


    if not all(key in yf_data.keys() for key in forecast_earnings_keys):
        # Handle the case where one or more keys are missing
        EPS_12month_projected = 0
    else:
        EPS_12month_projected = yf_data['forwardEps'] # 代表了截止下一个日历年结束的EPS, next year forecasted EPS


    # US Treasury section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            US_T_10yrs_df = pd.DataFrame(value)
            US_T_10yrs_df['value'] = pd.to_numeric(US_T_10yrs_df['value'], errors='coerce') # change dataframe value to numeric data
            US_T_10yrs_YTM = US_T_10yrs_df['value'][0]



    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    

    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')

    
            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])

            qtr_balancesheet_df['BVPS_latest'] = round(
                qtr_balancesheet_df.tail(1)['totalShareholderEquity'].sum() / qtr_balancesheet_df.tail(1)['commonStockSharesOutstanding']
                ,2
                )



    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100


    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']





    # Consolidated section
    stock_consolidate_df = Daily_stock_df.head(window_days)


    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr

        if i == max(stock_consolidate_df.index):
            EPS_nextQtr_projected = latest_projected_EPS + past_3_qtrs_EPS['reportedEPS'].values.sum()  # This metrics is the past 3 qtrs post EPS + 1 projected EPS
        else:
            continue

        stock_consolidate_df["EPS_nextQtr"] = latest_projected_EPS
        stock_consolidate_df["EPS_nextQtr_TTM"] = EPS_nextQtr_projected


    

    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_1yr_forward"] = round(PE_12month_projected, 2)
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数

    stock_consolidate_df["relative_valuation_nextQuater_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextQuater_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_nextQtr_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextQuater_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextQuater_projected_+"][0], stock_consolidate_df["relative_valuation_nextQuater_projected_-"][0]])).round(2) #这个是根据3 qtrs post EPS + 1 projected EPS 得出PE估值的价格中位数

    stock_consolidate_df[f"{window_days}_price_min"] = stock_consolidate_df["stock_price"].min().round(2)
    stock_consolidate_df[f"{window_days}_price_max"] = stock_consolidate_df["stock_price"].max().round(2)
    stock_consolidate_df[f"{window_days}_price_avg"] = stock_consolidate_df["stock_price"].mean().round(2)
    stock_consolidate_df[f"{window_days}_price_std"] = np.std(stock_consolidate_df["stock_price"]).round(2)

    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_avg"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_avg"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_std"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_std"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_+"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_+"].values[0]
    stock_consolidate_df[f"PE_{PE_yr_range-1}yr_volatility_-"] = annualEPS_df[f"PE_{PE_yr_range-1}yr_volatility_-"].values[0]


    stock_consolidate_df["relative_valuation_nextYear_projected_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_nextYear_projected_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * EPS_12month_projected).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_nextYear_projected_median"] = (np.median([stock_consolidate_df["relative_valuation_nextYear_projected_+"][0], stock_consolidate_df["relative_valuation_nextYear_projected_-"][0]])).round(2) #这个是根据next year projected EPS 得出PE估值的价格中位数

    stock_consolidate_df["PEG_next12months"] = PEG_12month_projected
    stock_consolidate_df["EPS_next12months"] = EPS_12month_projected
    stock_consolidate_df["PEG_TTM"] = (stock_consolidate_df["PE_TTM"] / (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100)).round(2) # 这个是截止下一年的EPS growth rate所得出的PEG ratio, <1是undervalue的表现
    
    stock_consolidate_df["EPS_nextYr_growthRate"] = (((EPS_12month_projected - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)
    stock_consolidate_df["EPS_nextQtr_growthRate"] = (((stock_consolidate_df["EPS_nextQtr_TTM"] - stock_consolidate_df["EPS_TTM"]) / stock_consolidate_df["EPS_TTM"]) * 100).round(2)

    stock_consolidate_df["EarningYield_TTM"] = ((stock_consolidate_df["EPS_TTM"] / stock_consolidate_df["stock_price"]) * 100).round(2)
    stock_consolidate_df["ERP_TTM"] = stock_consolidate_df["EarningYield_TTM"] - US_T_10yrs_YTM # it means a comparison between equity return and 10 years risk free, usually ERP >= 3 for short term invest at least, ERP >= 5 for long term invest 

    stock_consolidate_df['latest_qtr_liquidation_mktcap_ratio_%'] = stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'].values[-1]
    stock_consolidate_df['BVPS_latest'] = qtr_balancesheet_df['BVPS_latest'].values[-1]

    stock_consolidate_df['MA200_slope'] = Daily_stock_df['MA200_slope']

    stock_consolidate_df['FCF_per_share_TTM'] = round((qtr_cashflow_df['operatingCashflow'] - qtr_cashflow_df['capitalExpenditures']).tail(4).sum() / qtr_balancesheet_df['commonStockSharesOutstanding'].values[-1], 2)
    stock_consolidate_df['PFCF_TTM'] = round(stock_consolidate_df['stock_price'][0]  / stock_consolidate_df['FCF_per_share_TTM'], 2)
    stock_consolidate_df['FCF_yield_TTM'] = round((stock_consolidate_df['FCF_per_share_TTM'] / stock_consolidate_df['stock_price'][0]) * 100, 2) # % return on cash of every single dollar you spent on stock price, the higher the better


    # filter conditions
    conditions = [
    (stock_consolidate_df["stock_price"] < stock_consolidate_df["relative_valuation_TTM_-"]),
    (stock_consolidate_df["stock_price"] > stock_consolidate_df["relative_valuation_TTM_+"]),
    ((stock_consolidate_df["stock_price"] >= stock_consolidate_df["relative_valuation_TTM_-"]) & (stock_consolidate_df["stock_price"] <= stock_consolidate_df["relative_valuation_TTM_+"])),
    ]

    categories = [
        'undervalued'
        ,'overvalued'
        ,'fair'
        ]

    # This KPI assess if the current stock price is under/over/fair to the current relative valuation
    stock_consolidate_df["curr_assessment"] = None

    for condition, category in zip(conditions, categories):
        stock_consolidate_df.loc[condition, "price_valuation_assessment"] = category





    # Append key-value pairs to the dictionary
    selected_cols = [
    "Ticker"
    ,"stock_price"
    ,"EPS_currentQtr"
    ,"EPS_nextQtr"
    ,"EPS_TTM"
    ,"EPS_nextQtr_TTM"
    ,"EPS_next12months"
    ,"PE_1yr_forward"
    ,"PE_TTM"
    ,"PE_TTM_avg"
    ,"PE_TTM_volatility_+"
    ,"PE_TTM_volatility_-"
    ,f"PE_{PE_yr_range-1}yr_avg"
    ,f"PE_{PE_yr_range-1}yr_volatility_+"
    ,f"PE_{PE_yr_range-1}yr_volatility_-"
    ,"relative_valuation_TTM_+"
    ,"relative_valuation_TTM_-"
    ,"relative_valuation_TTM_median"
    ,"relative_valuation_nextQuater_projected_+"
    ,"relative_valuation_nextQuater_projected_-"
    ,"relative_valuation_nextQuater_projected_median"
    ,"relative_valuation_nextYear_projected_+"
    ,"relative_valuation_nextYear_projected_-"
    ,"relative_valuation_nextYear_projected_median"
    ,"price_valuation_assessment"
    ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    ,"PEG_next12months"
    ,"PEG_TTM"
    ,"EarningYield_TTM"
    ,"ERP_TTM"
    ,"latest_qtr_liquidation_mktcap_ratio_%"
    ,"BVPS_latest"
    ,"PFCF_TTM"
    ,"FCF_yield_TTM" 
    ,"MA200_slope"
    ]


    # store each stock info as pd into dictionary
    ticker_dict_pd[symbol] = stock_consolidate_df[selected_cols]
    # transfer pandas dataframe to json format, and each stock info into dictionary
    ticker_dict_json[symbol] = stock_consolidate_df[selected_cols].to_dict()

    if j == 0:
        # screener df creation
        # screener df will store each stock's consolidate df's first row and union them together for screening purposee
        ticker_screen_df = pd.DataFrame(
            columns=selected_cols
        ) 
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]

        # Insert rows into the DataFrame
        ticker_screen_df.loc[j] = stock_consolidate_df_values

    else:
        stock_consolidate_df_values = stock_consolidate_df[selected_cols].values[0]
        ticker_screen_df.loc[j] = stock_consolidate_df_values


# ticker screen df consolidated metrics
ticker_screen_df['Industry_PE_TTM_avg'] = round(ticker_screen_df['PE_TTM'].mean(), 2)

0 GOOG
1 KR


In [80]:
ticker_dict_pd['KR']

,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_next12months,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,PEG_next12months,PEG_TTM,EarningYield_TTM,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,BVPS_latest,PFCF_TTM,FCF_yield_TTM,MA200_slope
2025-01-06,KR,60.62,0.98,1.13,4.68,4.47,4.58,13.19,12.95,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,fair,-4.49,-2.14,2.76,-6.06,7.72,3.12,21.33,17.84,20.07,4.98,0.044298
2025-01-03,KR,61.46,0.98,1.13,4.68,4.47,4.58,13.19,13.13,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.14,7.61,3.01,21.33,17.84,20.07,4.98,0.046860
2025-01-02,KR,61.94,0.98,1.13,4.68,4.47,4.58,13.19,13.24,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.20,7.56,2.96,21.33,17.84,20.07,4.98,0.049303
2024-12-31,KR,61.15,0.98,1.13,4.68,4.47,4.58,13.19,13.07,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.12,7.65,3.05,21.33,17.84,20.07,4.98,0.051579
2024-12-30,KR,61.23,0.98,1.13,4.68,4.47,4.58,13.19,13.08,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.12,7.64,3.04,21.33,17.84,20.07,4.98,0.053728
2024-12-27,KR,62.34,0.98,1.13,4.68,4.47,4.58,13.19,13.32,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.23,7.51,2.91,21.33,17.84,20.07,4.98,0.055716
2024-12-26,KR,62.76,0.98,1.13,4.68,4.47,4.58,13.19,13.41,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.28,7.46,2.86,21.33,17.84,20.07,4.98,0.057682
2024-12-24,KR,61.83,0.98,1.13,4.68,4.47,4.58,13.19,13.21,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.18,7.57,2.97,21.33,17.84,20.07,4.98,0.059564
2024-12-23,KR,60.96,0.98,1.13,4.68,4.47,4.58,13.19,13.03,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.10,7.68,3.08,21.33,17.84,20.07,4.98,0.061375
2024-12-20,KR,61.85,0.98,1.13,4.68,4.47,4.58,13.19,13.22,12.38,12.99,11.77,12.85,14.01,11.69,60.79,55.08,57.94,58.07,52.61,55.34,59.49,53.91,56.7,overvalued,-4.49,-2.14,2.76,-6.19,7.57,2.97,21.33,17.84,20.07,4.98,0.063008


In [8]:
ticker_screen_df = ticker_screen_df.sort_values(by=['FCF_yield_TTM']
                                                ,ascending=False)

selected_cols = [
    'Ticker'
    ,'EPS_TTM'
    ,'PE_TTM'
    ,'PE_TTM_avg'
    ,'price_valuation_assessment'
    # ,"EPS_nextQtr_growthRate"
    ,"EPS_nextYr_growthRate"
    ,"PEG_next12months"
    # ,"PEG_TTM"
    ,'ERP_TTM'
    ,'latest_qtr_liquidation_mktcap_ratio_%'
    ,'PFCF_TTM'
    ,'FCF_yield_TTM'
    ,'Industry_PE_TTM_avg'
    ,'BVPS_latest'
    ,'stock_price'
]

ticker_screen_df[selected_cols]

,Ticker,EPS_TTM,PE_TTM,PE_TTM_avg,price_valuation_assessment,EPS_nextYr_growthRate,PEG_next12months,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,PFCF_TTM,FCF_yield_TTM,Industry_PE_TTM_avg,BVPS_latest,stock_price
1,LPG,6.14,4.06,4.3,fair,-36.32,-1.02,20.08,NaN,3.62,27.66,12.03,25.42,24.91
2,AVT,4.64,11.22,10.81,fair,25.43,2.53,4.34,93.18,7.0,14.29,12.03,57.19,52.08
3,CE,9.14,7.24,11.3,undervalued,5.25,4.42,9.25,-49.54,8.57,11.67,12.03,66.57,66.13
0,GOOG,7.54,25.61,23.76,overvalued,18.7,1.25,-0.67,11.98,42.45,2.36,12.03,25.61,193.13


In [ ]:
ticker_screen_df

# [ticker_screen_df['price_valuation_assessment'] == 'undervalued']


,Ticker,stock_price,EPS_currentQtr,EPS_nextQtr,EPS_TTM,EPS_nextQtr_TTM,EPS_next12months,PE_1yr_forward,PE_TTM,PE_TTM_avg,PE_TTM_volatility_+,PE_TTM_volatility_-,PE_5yr_avg,PE_5yr_volatility_+,PE_5yr_volatility_-,relative_valuation_TTM_+,relative_valuation_TTM_-,relative_valuation_TTM_median,relative_valuation_nextQuater_projected_+,relative_valuation_nextQuater_projected_-,relative_valuation_nextQuater_projected_median,relative_valuation_nextYear_projected_+,relative_valuation_nextYear_projected_-,relative_valuation_nextYear_projected_median,price_valuation_assessment,EPS_nextQtr_growthRate,EPS_nextYr_growthRate,PEG_next12months,PEG_TTM,EarningYield_TTM,ERP_TTM,latest_qtr_liquidation_mktcap_ratio_%,BVPS_latest,PFCF_TTM,FCF_yield_TTM,MA200_slope,Industry_PE_TTM_avg
2,AXP,269.11,3.49,3.29,12.74,12.73,14.94,18.59,21.12,19.69,20.57,18.81,18.08,24.49,11.67,262.06,239.64,250.85,261.86,239.45,250.66,307.32,281.02,294.17,overvalued,-0.08,17.27,2.09,1.22,4.73,0.7,NaN,41.49,8.22,12.17,0.479385,28.82
0,V,274.96,2.42,2.58,9.67,9.92,11.07,24.51,28.43,28.37,29.28,27.46,33.45,38.96,27.94,283.14,265.54,274.34,290.46,272.4,281.43,324.13,303.98,314.06,fair,2.59,14.48,1.78,1.96,3.52,-0.51,-1.0,21.65,26.62,3.76,0.114991,28.82
1,MA,497.06,3.59,3.73,13.47,13.81,16.63,29.59,36.9,35.24,36.22,34.26,38.85,47.45,30.25,487.88,461.48,474.68,500.2,473.13,486.66,602.34,569.74,586.04,overvalued,2.52,23.46,1.78,1.57,2.71,-1.32,-0.93,7.99,41.98,2.38,0.395689,28.82


# Price EPS chart

In [105]:
# Parameters section
alpha_vantage_api_key = API_KEY


ticker_symbols = [
    'CCA'
]



In [106]:
# Daily quote section
for j, symbol in enumerate(ticker_symbols):

    print(j, symbol)
    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df = pd.DataFrame()
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()


    # Daily quote section
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={alpha_vantage_api_key}&outputsize=full'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Time Series (Daily)':


            selected_cols = [
                '4. close'
            ]

            Daily_stock_df = pd.DataFrame(value).transpose()[selected_cols] # tranpose the dataframe and sub select selected cols

            # Rename columns
            Daily_stock_df.rename(
                columns={
                    '4. close': 'stock_price'
                    }
                ,inplace=True
                )
            
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
            Daily_stock_df["stock_price"] = Daily_stock_df["stock_price"].round(2)
            Daily_stock_df.index = pd.to_datetime(Daily_stock_df.index)


    for date_i in Daily_stock_df.index.date:
        for date_j in stock_split_record_df['effective_date'].dt.date:
            if date_i == date_j:

                # stock price to divided the split factor
                Daily_stock_df.loc[Daily_stock_df.index.date < date_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.date == date_j].values[0])




    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year


            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)



        if key == 'quarterlyEarnings':

            selected_cols = [
                'reportedDate'
                ,'reportedEPS'
            ]

            qtrEPS_df = pd.DataFrame(value)[selected_cols] # tranpose the dataframe and sub select selected cols
            qtrEPS_df['reportedDate'] = pd.to_datetime(qtrEPS_df['reportedDate'])

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    qtrEPS_df[col] = qtrEPS_df[col].astype(str).apply(lambda x: float(x) if x not in [None, 'None', 'nan', 'NaN'] else float(0))
                else:
                    continue



    # Consolidated section
    stock_consolidate_df = Daily_stock_df
    stock_consolidate_df_date = stock_consolidate_df.index
    for i in stock_consolidate_df_date:
                
        # Filter the DataFrame to include only dates(index) less than or equal to the target date
        filtered_qtrEPS_df = qtrEPS_df[qtrEPS_df['reportedDate'] < i]

        # Select the first four rows from the past_qtrs_EPS
        past_4_qtrs_EPS = filtered_qtrEPS_df.head(4) 
        past_3_qtrs_EPS = filtered_qtrEPS_df.head(3)
        past_1_qtr_EPS = filtered_qtrEPS_df.head(1)

        # Calculate the sum of the numeric values in the selected rows
        EPS_TTM = past_4_qtrs_EPS['reportedEPS'].values.sum()
        EPS_curr_qtr = past_1_qtr_EPS['reportedEPS'].values.sum()

        # assign each index row with the EPS_TTM
        stock_consolidate_df.loc[i, "EPS_TTM"] = EPS_TTM
        stock_consolidate_df.loc[i, "EPS_currentQtr"] = EPS_curr_qtr


    # stock's stats
    stock_consolidate_df["Ticker"] = symbol
    stock_consolidate_df["PE_TTM"] = (stock_consolidate_df["stock_price"] / stock_consolidate_df["EPS_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_avg"] = stock_consolidate_df["PE_TTM"].mean().round(2)
    stock_consolidate_df["PE_TTM_std"] = np.std(stock_consolidate_df["PE_TTM"]).round(2)
    stock_consolidate_df["PE_TTM_volatility_+"] = (stock_consolidate_df["PE_TTM_avg"] + stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围上限
    stock_consolidate_df["PE_TTM_volatility_-"] = (stock_consolidate_df["PE_TTM_avg"] - stock_consolidate_df["PE_TTM_std"]).round(2) # 这个是PE的波动范围下限

    stock_consolidate_df["relative_valuation_TTM_+"] = (stock_consolidate_df["PE_TTM_volatility_+"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格上限
    stock_consolidate_df["relative_valuation_TTM_-"] = (stock_consolidate_df["PE_TTM_volatility_-"] * stock_consolidate_df["EPS_TTM"]).round(2) # 这个是relative valuation的价格下限
    stock_consolidate_df["relative_valuation_TTM_median"] = (np.median([stock_consolidate_df["relative_valuation_TTM_+"][0], stock_consolidate_df["relative_valuation_TTM_-"][0]])).round(2) #这个是根据最新TTM PE估值的价格中位数


0 CCA


In [107]:
import plotly.graph_objects as go


# Create the figure
fig = go.Figure()

# Add stock_price on primary y-axis (left)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["stock_price"],
    mode='lines',
    line=dict(color='black'),
    name='Stock Price',
    yaxis="y1"
))

# Add EPS_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["EPS_TTM"],
    mode='lines',
    fill='tonexty',  # Shadow effect
    line=dict(color='green'),
    name='EPS_TTM',
    yaxis="y2"
))

# Add PE_TTM on secondary y-axis (right)
fig.add_trace(go.Scatter(
    x=stock_consolidate_df.index,
    y=stock_consolidate_df["PE_TTM"],
    mode='lines',
    line=dict(color='blue', dash='dot'),
    name='PE_TTM',
    yaxis="y2"
))

# Update layout to remove grid and configure dual y-axes
fig.update_layout(
    title="Stock Metrics Over Time",
    xaxis=dict(title="Date", showgrid=False),
    yaxis=dict(
        title="Stock Price",
        showgrid=False,
        titlefont=dict(color="black"),
        tickfont=dict(color="black")
    ),
    yaxis2=dict(
        title="EPS_TTM & PE_TTM",
        overlaying="y",  # Overlay with y1
        side="right",
        showgrid=False,
        titlefont=dict(color="blue"),
        tickfont=dict(color="blue")
    ),
    legend_title="Metrics",
    template="plotly_white",
    plot_bgcolor='white'  # Background color
)

# Show the plot
fig.show()

# Financial Statement

In [84]:
stock_financial_dict = {} # the dict will store ticker as key, the string value of 'annual_df' and 'qtr_df' as nested key, actual json as value

ticker_symbols = [
    'KR'
    ]
# ticker_symbols = electric_utility_symbols
PE_yr_range = 15 # test for 'X' years PE 


# storage process
for j, symbol in enumerate(ticker_symbols):
    print(j, symbol)
    # COMPANY OVERVIEW
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    
    for key, value in data.items():
        if key  == 'MarketCapitalization':
            stock_mkt_cap = float(value)



    # STOCK SPLIT FACTOR section
    url = f'https://www.alphavantage.co/query?function=SPLITS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'data':
            if len(value) > 0:
                stock_split_record_df = pd.DataFrame(value)
                stock_split_record_df['split_factor'] = pd.to_numeric(stock_split_record_df['split_factor'], errors='coerce') # change split_factor series to numeric data
                stock_split_record_df['effective_date'] = pd.to_datetime(stock_split_record_df['effective_date'])
            else:
                stock_split_record_df['split_factor'] = 1
                stock_split_record_df['effective_date'] = datetime.today()



    # Monthly quote section
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'Monthly Time Series':
            Monthly_stock_df = pd.DataFrame(value)


    Monthly_stock_df = Monthly_stock_df.transpose()
    Monthly_stock_df.index = pd.to_datetime(Monthly_stock_df.index)


    filter_1 = (Monthly_stock_df.index.year.isin(range((datetime.today().year - PE_yr_range) ,datetime.today().year)))
    filter_2 = (Monthly_stock_df.index.month == 12) # month = 12 to get the year end closing price

    selected_cols = [
        '4. close'
    ]

    Monthly_stock_df = Monthly_stock_df[
        filter_1
        & filter_2
    ][selected_cols]

    # Rename columns
    Monthly_stock_df.rename(
        columns={
            '4. close': 'stock_price'
            }
        ,inplace=True
        )

    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].astype(str).apply(lambda x: float(x))
    Monthly_stock_df["stock_price"] = Monthly_stock_df["stock_price"].round(2)

    # modify stock price based on stock split
    for year_i in Monthly_stock_df.index.year:
        for year_j in stock_split_record_df['effective_date'].dt.year:
            if year_i == year_j:

                # stock price to divided the split factor
                Monthly_stock_df.loc[Monthly_stock_df.index.year < year_j, 'stock_price'] /= (stock_split_record_df['split_factor'][stock_split_record_df['effective_date'].dt.year == year_j].values[0])
    


    # Earning section
    # past earnings from alpha vintage API
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():
        if key == 'annualEarnings':

            selected_cols = [
                'fiscalDateEnding'
                ,'reportedEPS'
            ]

            annualEPS_df = pd.DataFrame(value) # tranpose the dataframe and sub select selected cols


            annualEPS_df['fiscalDateEnding'] = pd.to_datetime(annualEPS_df['fiscalDateEnding']).dt.year

            annualEPS_df = annualEPS_df[
                annualEPS_df['fiscalDateEnding'].isin(
                    range(
                        (datetime.today().year - PE_yr_range) 
                        ,datetime.today().year
                            )
                            )
                            ]

            # Convert the column to decimal type
            for col in selected_cols:
                if col in ['reportedEPS']:
                    annualEPS_df[f'{col}'] = annualEPS_df[f'{col}'].astype(str).apply(lambda x: float(x))

                else:
                    continue
            # clean annualEPS_df
            annualEPS_df = annualEPS_df.sort_values('reportedEPS', ascending=False).drop_duplicates('fiscalDateEnding')
            annualEPS_df = annualEPS_df.sort_values('fiscalDateEnding', ascending=False).reset_index(drop=True)

            # calculate PE
            annualEPS_df["PE"] = Monthly_stock_df["stock_price"].values / annualEPS_df['reportedEPS'].values




    # INCOME STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()


    for key, value in data.items():
        if key == 'annualReports':
            annual_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_income_df = annual_income_df.sort_values('fiscalDateEnding', ascending=True)

            # annual measurements cols
            annual_income_metric_cols = annual_income_df.columns[1:]

            # Calculate quarter-over-quarter change percentage
            for column in annual_income_metric_cols:  # Exclude the 'fiscalDateEnding' column
                annual_income_df[column] = pd.to_numeric(annual_income_df[column], errors='coerce')
                annual_income_df[f'{column}_YoY'] = annual_income_df[column].pct_change() * 100 
            
            annual_income_YoY_metric_cols = [col for col in annual_income_df.columns if 'YoY' in col]

            # ratios of income statement calculation
            annual_income_df['gross_margin_%'] = (annual_income_df['grossProfit'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['operating_margin_%'] = (annual_income_df['operatingIncome'] / annual_income_df['totalRevenue']) * 100
            annual_income_df['net_margin_%'] = (annual_income_df['netIncome'] / annual_income_df['totalRevenue']) * 100



        if key == 'quarterlyReports':
            qtr_income_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_income_df = qtr_income_df.sort_values('fiscalDateEnding', ascending=True)

            # qtr measurements cols
            qtr_income_metric_cols = qtr_income_df.columns[1:]
            
            # Calculate quarter-over-quarter change percentage
            for column in qtr_income_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_income_df[column] = pd.to_numeric(qtr_income_df[column], errors='coerce')
                qtr_income_df[f'{column}_QoQ'] = qtr_income_df[column].pct_change() * 100

            qtr_income_QoQ_metric_cols = [col for col in qtr_income_df.columns if 'QoQ' in col]

            # ratios of income statement calculation
            qtr_income_df['gross_margin_%'] = (qtr_income_df['grossProfit'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['operating_margin_%'] = (qtr_income_df['operatingIncome'] / qtr_income_df['totalRevenue']) * 100
            qtr_income_df['net_margin_%'] = (qtr_income_df['netIncome'] / qtr_income_df['totalRevenue']) * 100

    income_ratio_cols = [
        'gross_margin_%'
        ,'operating_margin_%'
        ,'net_margin_%'
        ,'netIncome'
        ]
    
    
    # BALANCESHEET
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_balancesheet_df = annual_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_balancesheet_df[column] = pd.to_numeric(annual_balancesheet_df[column], errors='coerce')
                annual_balancesheet_df[f'{column}_YoY'] = annual_balancesheet_df[column].pct_change() * 100

            annual_balancesheet_df['current_ratio'] = (annual_balancesheet_df['totalCurrentAssets'] / annual_balancesheet_df['totalCurrentLiabilities'])
            annual_balancesheet_df['working_capital'] = annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['totalCurrentLiabilities']
            annual_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (annual_balancesheet_df['longTermDebt'] / annual_balancesheet_df['working_capital'])
            annual_balancesheet_df['debtEquity_ratio'] = (annual_balancesheet_df['totalLiabilities'] / annual_balancesheet_df['totalShareholderEquity'])
            annual_balancesheet_df['quick_ratio'] = ((annual_balancesheet_df['totalCurrentAssets'] - annual_balancesheet_df['inventory']) / annual_balancesheet_df['totalCurrentLiabilities'])

            
        if key == 'quarterlyReports':
            qtr_balancesheet_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_balancesheet_df = qtr_balancesheet_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_balancesheet_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_balancesheet_df[column] = pd.to_numeric(qtr_balancesheet_df[column], errors='coerce')
                qtr_balancesheet_df[f'{column}_YoY'] = qtr_balancesheet_df[column].pct_change() * 100

            qtr_balancesheet_df['current_ratio'] = (qtr_balancesheet_df['totalCurrentAssets'] / qtr_balancesheet_df['totalCurrentLiabilities'])
            qtr_balancesheet_df['working_capital'] = qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['totalCurrentLiabilities']
            qtr_balancesheet_df['longTermDebt_to_workingCp_ratio'] = (qtr_balancesheet_df['longTermDebt'] / qtr_balancesheet_df['working_capital'])
            qtr_balancesheet_df['debtEquity_ratio'] = (qtr_balancesheet_df['totalLiabilities'] / qtr_balancesheet_df['totalShareholderEquity'])
            qtr_balancesheet_df['quick_ratio'] = ((qtr_balancesheet_df['totalCurrentAssets'] - qtr_balancesheet_df['inventory']) / qtr_balancesheet_df['totalCurrentLiabilities'])


    balancesheet_ratio_cols = [
        'current_ratio'
        ,'working_capital'
        ,'longTermDebt_to_workingCp_ratio'
        ,'debtEquity_ratio'
        ,'quick_ratio'
        ,'totalShareholderEquity'
    ]


    # CASHFLOW STATEMENT
    # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={alpha_vantage_api_key}'
    r = requests.get(url)
    data = r.json()

    for key, value in data.items():

        if key == 'annualReports':
            annual_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            annual_cashflow_df = annual_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in annual_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                annual_cashflow_df[column] = pd.to_numeric(annual_cashflow_df[column], errors='coerce')
                annual_cashflow_df[f'{column}_YoY'] = annual_cashflow_df[column].pct_change() * 100

            
        if key == 'quarterlyReports':
            qtr_cashflow_df = pd.DataFrame(value).drop(['reportedCurrency'], axis=1)
            qtr_cashflow_df = qtr_cashflow_df.sort_values('fiscalDateEnding', ascending=True)

            for column in qtr_cashflow_df.columns[1:]:  # Exclude the 'fiscalDateEnding' column
                qtr_cashflow_df[column] = pd.to_numeric(qtr_cashflow_df[column], errors='coerce')
                qtr_cashflow_df[f'{column}_YoY'] = qtr_cashflow_df[column].pct_change() * 100



    cashflow_ratio_cols = [
        'operatingCashflow'
        ,'capitalExpenditures'
        ,'cashflowFromInvestment'
        ,'cashflowFromFinancing'
        ,'netIncome'
    ]


    ####################
    ### Consolidated ###
    ####################
    stock_ratios_annual_consolidate_df = pd.DataFrame()
    stock_ratios_qtr_consolidate_df = pd.DataFrame()
    stock_PE_annual_consolidate_df = pd.DataFrame()

    stock_ratios_annual_consolidate_df[income_ratio_cols] = annual_income_df[income_ratio_cols]
    stock_ratios_annual_consolidate_df[balancesheet_ratio_cols] = annual_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_annual_consolidate_df[cashflow_ratio_cols] = annual_cashflow_df[cashflow_ratio_cols]

    stock_ratios_qtr_consolidate_df[income_ratio_cols] = qtr_income_df[income_ratio_cols]
    stock_ratios_qtr_consolidate_df[balancesheet_ratio_cols] = qtr_balancesheet_df[balancesheet_ratio_cols]
    stock_ratios_qtr_consolidate_df[cashflow_ratio_cols] = qtr_cashflow_df[cashflow_ratio_cols]

    # calculating new consolidated metrics 
    # annual df
    stock_ratios_annual_consolidate_df['ROE_%'] = (annual_income_df['netIncome'] / annual_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_annual_consolidate_df['liquidation_value'] = ((annual_balancesheet_df['totalAssets'] - annual_balancesheet_df['intangibleAssets']) - annual_balancesheet_df['totalLiabilities'])
    stock_ratios_annual_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_annual_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_annual_consolidate_df['fiscalDateEnding'] = annual_balancesheet_df['fiscalDateEnding']

    stock_ratios_annual_consolidate_df['PE'] = annualEPS_df["PE"]
    stock_ratios_annual_consolidate_df['reportedEPS'] = annualEPS_df["reportedEPS"]

    # YoY %
    stock_ratios_annual_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_annual_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_annual_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_annual_consolidate_df["net_margin_%"].pct_change() * 100
    stock_ratios_annual_consolidate_df['YoY_reportedEPS_%'] = stock_ratios_annual_consolidate_df["reportedEPS"].pct_change() * 100 # yearly EPS 的增速



    # qtr df
    stock_ratios_qtr_consolidate_df['ROE_%'] = (qtr_income_df['netIncome'] / qtr_balancesheet_df['totalShareholderEquity']) * 100
    stock_ratios_qtr_consolidate_df['liquidation_value'] = ((qtr_balancesheet_df['totalAssets'] - qtr_balancesheet_df['intangibleAssets']) - qtr_balancesheet_df['totalLiabilities'])
    stock_ratios_qtr_consolidate_df['liquidation_mktcap_ratio_%'] = round((stock_ratios_qtr_consolidate_df['liquidation_value'] / stock_mkt_cap) * 100, 2)
    stock_ratios_qtr_consolidate_df['fiscalDateEnding'] = qtr_balancesheet_df['fiscalDateEnding']
    
    # QoQ %
    stock_ratios_qtr_consolidate_df['QoQ_gross_margin_%_%'] = stock_ratios_qtr_consolidate_df["gross_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_operating_margin_%_%'] = stock_ratios_qtr_consolidate_df["operating_margin_%"].pct_change() * 100
    stock_ratios_qtr_consolidate_df['QoQ_net_margin_%_%'] = stock_ratios_qtr_consolidate_df["net_margin_%"].pct_change() * 100




    # annual PE df
    if stock_PE_annual_consolidate_df.empty:
        stock_PE_annual_consolidate_df['fiscalDateEnding'] = annualEPS_df['fiscalDateEnding']

    stock_PE_annual_consolidate_df[f'{symbol}_PE'] = annualEPS_df["PE"].round(2)



    # reformat the columns order
    stock_ratios_annual_consolidate_df = stock_ratios_annual_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_annual_consolidate_df.columns if col != 'fiscalDateEnding']]
    stock_ratios_qtr_consolidate_df = stock_ratios_qtr_consolidate_df[['fiscalDateEnding'] + [col for col in stock_ratios_qtr_consolidate_df.columns if col != 'fiscalDateEnding']]




    # store the stock, dataframe value pair to the dictionary
    # transfer the pandas df to json
    stock_ratios_annual_consolidate_json = stock_ratios_annual_consolidate_df.to_dict()
    stock_ratios_qtr_consolidate_json = stock_ratios_qtr_consolidate_df.to_dict()

    stock_financial_dict[symbol] = {
    'annual': stock_ratios_annual_consolidate_json
    ,'qtr': stock_ratios_qtr_consolidate_json
    }

0 KR


In [85]:
qtr_balancesheet_df

,fiscalDateEnding,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,accumulatedDepreciationAmortizationPPE,intangibleAssets,intangibleAssetsExcludingGoodwill,goodwill,investments,longTermInvestments,shortTermInvestments,otherCurrentAssets,otherNonCurrentAssets,totalLiabilities,totalCurrentLiabilities,currentAccountsPayable,deferredRevenue,currentDebt,shortTermDebt,totalNonCurrentLiabilities,capitalLeaseObligations,longTermDebt,currentLongTermDebt,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,totalAssets_YoY,totalCurrentAssets_YoY,cashAndCashEquivalentsAtCarryingValue_YoY,cashAndShortTermInvestments_YoY,inventory_YoY,currentNetReceivables_YoY,totalNonCurrentAssets_YoY,propertyPlantEquipment_YoY,accumulatedDepreciationAmortizationPPE_YoY,intangibleAssets_YoY,intangibleAssetsExcludingGoodwill_YoY,goodwill_YoY,investments_YoY,longTermInvestments_YoY,shortTermInvestments_YoY,otherCurrentAssets_YoY,otherNonCurrentAssets_YoY,totalLiabilities_YoY,totalCurrentLiabilities_YoY,currentAccountsPayable_YoY,deferredRevenue_YoY,currentDebt_YoY,shortTermDebt_YoY,totalNonCurrentLiabilities_YoY,capitalLeaseObligations_YoY,longTermDebt_YoY,currentLongTermDebt_YoY,longTermDebtNoncurrent_YoY,shortLongTermDebtTotal_YoY,otherCurrentLiabilities_YoY,otherNonCurrentLiabilities_YoY,totalShareholderEquity_YoY,treasuryStock_YoY,retainedEarnings_YoY,commonStock_YoY,commonStockSharesOutstanding_YoY,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio
57,2009-11-07,23084000000,7530000000,517000000,8.265000e+08,5.245000e+09,817000000,1.555420e+10,13818000000,NaN,1.158000e+09,1.158000e+09,1158000000,NaN,NaN,3.594000e+08,9.510000e+08,578000000,18232000000,8164000000,NaN,NaN,NaN,5.772000e+08,1.006780e+10,NaN,7.476000e+09,NaN,NaN,8.053700e+09,2290000000,1242000000,4783000000,6.145000e+09,7123000000,9.570000e+08,646000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.922342,-634000000,-11.791798,3.811834,0.279887
56,2010-01-30,23126000000,7483000000,424000000,4.240000e+08,4.935000e+09,909000000,1.564300e+10,13929000000,1.174400e+10,NaN,NaN,1158000000,NaN,NaN,6.540000e+08,5.610000e+08,556000000,18200000000,7727000000,NaN,NaN,NaN,6.770000e+08,1.047300e+10,NaN,7.576000e+09,5.490000e+08,7.027000e+09,8.056000e+09,2118000000,1346000000,4852000000,6.238000e+09,7364000000,9.580000e+08,644000000,0.181944,-0.624170,-17.988395,-48.699335,-5.910391,11.260710,0.570907,0.803300,NaN,0.000000,0.000000,0.000000,NaN,NaN,81.969950,-41.009464,-3.806228,-0.175516,-5.352768,NaN,NaN,NaN,17.290367,4.024712,NaN,1.337614,NaN,NaN,0.028558,-7.510917,8.373591,1.442609,1.513426,3.383406,0.104493,-0.309598,0.968422,-244000000,-31.049180,3.751031,0.329753
55,2010-05-22,22881000000,7182000000,602000000,NaN,NaN,828000000,NaN,13976000000,NaN,NaN,NaN,1158000000,NaN,NaN,NaN,NaN,565000000,17775000000,7801000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2124000000,1342000000,5108000000,6.314000e+09,7676000000,9.590000e+08,641000000,-1.059414,-4.022451,41.981132,0.000000,0.000000,-8.910891,0.000000,0.337426,0.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,1.618705,-2.335165,0.957681,NaN,NaN,NaN,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.283286,-0.297177,5.276175,1.218339,4.236828,0.104384,-0.465839,0.920651,-619000000,NaN,3.479836,NaN
54,2010-08-14,23276000000,7546000000,1037000000,NaN,4.650000e+09,805000000,NaN,14003000000,NaN,NaN,NaN,1158000000,NaN,NaN,NaN,NaN,569000000,18109000000,7925000000,NaN,NaN,NaN,NaN,NaN,NaN,7.333000e+09,5.070000e+08,6.826000e+09,NaN,2345000000,1335000000,5163000000,6.433000e+09,7877000000,9.590000e+08,637000000,1.726323,5.068226,72.259136,0.000000,-

In [86]:
qtr_income_df

,fiscalDateEnding,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,investmentIncomeNet,netInterestIncome,interestIncome,interestExpense,nonInterestIncome,otherNonOperatingIncome,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,grossProfit_QoQ,totalRevenue_QoQ,costOfRevenue_QoQ,costofGoodsAndServicesSold_QoQ,operatingIncome_QoQ,sellingGeneralAndAdministrative_QoQ,researchAndDevelopment_QoQ,operatingExpenses_QoQ,investmentIncomeNet_QoQ,netInterestIncome_QoQ,interestIncome_QoQ,interestExpense_QoQ,nonInterestIncome_QoQ,otherNonOperatingIncome_QoQ,depreciation_QoQ,depreciationAndAmortization_QoQ,incomeBeforeTax_QoQ,incomeTaxExpense_QoQ,interestAndDebtExpense_QoQ,netIncomeFromContinuingOperations_QoQ,comprehensiveIncomeNetOfTax_QoQ,ebit_QoQ,ebitda_QoQ,netIncome_QoQ,gross_margin_%,operating_margin_%,net_margin_%
65,2008-08-16,NaN,NaN,14066000000,NaN,546000000,NaN,NaN,NaN,NaN,NaN,NaN,111000000,NaN,NaN,NaN,327000000.0,436000000,159000000,NaN,NaN,NaN,547000000,NaN,277000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,2008-11-08,4.028000e+09,1.758000e+10,13545000000,1.355200e+10,479000000,3.243000e+09,NaN,3.549000e+09,NaN,NaN,NaN,106000000,NaN,NaN,NaN,335000000.0,373000000,136000000,NaN,NaN,NaN,479000000,8.140000e+08,237000000,NaN,NaN,-3.703967,NaN,-12.271062,NaN,NaN,NaN,NaN,NaN,NaN,-4.504505,NaN,NaN,NaN,2.446483,-14.449541,-14.465409,NaN,NaN,NaN,-12.431444,NaN,-14.440433,22.912400,2.724687,1.348123
63,2009-01-31,4.172000e+09,1.726000e+10,13088000000,1.308800e+10,658000000,3.165000e+09,NaN,3.513000e+09,NaN,NaN,NaN,116000000,NaN,NaN,NaN,348000000.0,544000000,195000000,NaN,NaN,NaN,660000000,1.007000e+09,349000000,3.574975,-1.820250,-3.373939,-3.423849,37.369520,-2.405180,NaN,-1.014370e+00,NaN,NaN,NaN,9.433962,NaN,NaN,NaN,3.880597,45.844504,43.382353,NaN,NaN,NaN,37.787056,23.710074,47.257384,24.171495,3.812283,2.022016
62,2009-05-23,NaN,NaN,17266000000,NaN,844000000,NaN,NaN,NaN,NaN,NaN,NaN,163000000,NaN,NaN,NaN,NaN,685000000,250000000,NaN,NaN,NaN,848000000,NaN,435000000,0.000000,0.000000,31.922372,0.000000,28.267477,0.000000,NaN,0.000000e+00,NaN,NaN,NaN,40.517241,NaN,NaN,NaN,0.000000,25.919118,28.205128,NaN,NaN,NaN,28.484848,0.000000,24.641834,NaN,NaN,NaN
61,2009-08-15,3.967000e+09,1.761300e+10,13646000000,NaN,499000000,1.700000e+08,NaN,2.850000e+08,NaN,-115000000.0,NaN,115000000,NaN,NaN,NaN,348000000.0,388000000,133000000,115000000.0,2.510000e+08,2.530000e+08,503000000,8.510000e+08,255000000,-4.913710,2.045191,-20.966060,0.000000,-40.876777,-94.628752,NaN,-9.188728e+01,NaN,NaN,NaN,-29.447853,NaN,NaN,NaN,0.000000,-43.357664,-46.800000,NaN,NaN,NaN,-40.683962,-15.491559,-41.379310,22.523136,2.833135,1.447794
60,2009-11-07,3.895000e+09,1.755700e+10,13662000000,1.366660e+10,-758000000,1.710000e+08,NaN,1.389000e+09,NaN,-105000000.0,NaN,105000000,NaN,NaN,NaN,356000000.0,-862000000,13000000,105000000.0,-8.760000e+08,-8.700000e+08,-757000000,-4.010000e+08,-875000000,-1.814974,-0.317947,0.117250,4.420844,-251.903808,0.588235,NaN,3.873684e+02,NaN,-8.695652,NaN,-8.695652,NaN,NaN,NaN,2.298851,-322.164948,-90.225564,-8.695652,-449.003984,-443.873518,-250.497018,-147.121034,-443.137255,22.184884,-4.317366,-4.983767
59,2010-01-30,4.051000e+09,1.843500e+10,14384000000,1.437500e+10,506000000,1.460000e+08,NaN,2.650000e+08,NaN,-119000000.0,NaN,119000000,NaN,NaN,NaN,368000000.0,391000000,136000000,119000000.0,2.510000e+08,1.530000e+08,510000000,8.740000e+08,255000000,4.005135,5.000854,5.284731,5.183440,-166.754617,-14.619883,NaN,-8.092153e+01,NaN,13.333333,NaN,13.333333,NaN,NaN,NaN,3.370787,-145.359629,946.153846,13.333333,-128.652968,-117.586207,-167.371202,-317.955112,-129.142857,21.974505,2.744779,1.383238
58,2010-05-22,5.464000e+09,2.463200e+10,19155000000,

In [87]:
pd.DataFrame(stock_financial_dict['KR']['qtr']).tail(20)

,fiscalDateEnding,gross_margin_%,operating_margin_%,net_margin_%,netIncome,current_ratio,working_capital,longTermDebt_to_workingCp_ratio,debtEquity_ratio,quick_ratio,totalShareholderEquity,operatingCashflow,capitalExpenditures,cashflowFromInvestment,cashflowFromFinancing,ROE_%,liquidation_value,liquidation_mktcap_ratio_%,QoQ_gross_margin_%_%,QoQ_operating_margin_%_%,QoQ_net_margin_%_%
19,2019-11-09,21.729906,1.867631,1.137273,327000000,0.780918,-3.093000e+09,-4.098933,4.112731,0.255914,8.915000e+09,6.160000e+08,7.650000e+08,-7.870000e+08,2.500000e+07,3.667975,4.566000e+09,10.41,1.526587,104.682086,20.373606
18,2020-02-01,24.004834,3.204834,2.929305,1212000000,0.764586,-3.353000e+09,-3.953475,4.264473,0.267219,8.602000e+09,4.245000e+09,6.980000e+08,-6.890000e+08,-1.229000e+09,14.089747,4.431000e+09,10.10,10.469108,71.598859,157.572815
17,2020-05-23,22.438558,2.701456,2.698162,819000000,0.829868,-2.496000e+09,-5.049279,4.020308,NaN,9.356000e+09,1.160000e+09,6.450000e+08,-6.590000e+08,-4.070000e+08,8.753741,NaN,NaN,-6.524834,-15.706826,-7.890726
16,2020-11-07,22.646482,2.676218,2.132189,631000000,0.834311,-2.571000e+09,-4.912096,3.815392,0.352388,1.007000e+10,4.920000e+08,7.190000e+08,-7.000000e+08,-4.320000e+08,6.266137,5.956000e+09,13.58,0.926637,-0.934237,-20.976235
15,2021-01-30,22.660050,-0.515817,-0.251379,-77000000,0.813680,-2.863000e+09,-4.334614,4.084378,0.354028,9.576000e+09,9.180000e+08,8.030000e+08,-7.660000e+08,-6.450000e+08,-0.804094,5.477000e+09,12.49,0.059912,-119.274113,-111.789730
14,2021-05-22,22.376194,1.957066,0.340359,140000000,0.824786,-2.665000e+09,-4.774109,4.279598,NaN,9.249000e+09,2.256000e+09,8.200000e+08,-8.530000e+08,-7.810000e+08,1.513677,5.175000e+09,11.80,-1.252671,-479.410708,-235.396706
13,2021-08-14,21.068315,2.659693,1.480425,467000000,0.790361,-3.271000e+09,-3.888108,4.225372,NaN,9.278000e+09,8.670000e+08,4.990000e+08,-4.310000e+08,-5.200000e+08,5.033412,5.216000e+09,11.89,-5.844958,35.902027,334.959378
12,2021-11-06,21.084318,2.736013,1.522459,483000000,0.796523,-3.418000e+09,-3.580456,4.237630,0.348851,9.519000e+09,1.668000e+09,6.890000e+08,-6.750000e+08,-9.300000e+08,5.074062,5.461000e+09,12.45,0.075959,2.869509,2.839309
11,2022-01-29,18.095093,2.931794,1.716543,565000000,0.745819,-4.149000e+09,-2.830803,4.195620,0.330270,9.452000e+09,1.399000e+09,6.060000e+08,-6.520000e+08,-1.214000e+09,5.977571,5.411000e+09,12.34,-14.177480,7.155720,12.748063
10,2022-05-21,21.356054,3.387885,1.494721,664000000,0.756004,-3.962000e+09,-2.960878,4.218255,NaN,9.411000e+09,1.102000e+09,7.450000e+08,-7.230000e+08,-8.180000e+08,7.055573,5.389000e+09,12.29,18.021244,15.556708,-12.922569


In [88]:
# searching & merge process
YoY_EPS_screener = pd.DataFrame()

for symbol in stock_financial_dict.keys():
    annaul_df = pd.DataFrame(
        stock_financial_dict[symbol]['annual']
    )

    annaul_df = annaul_df[['fiscalDateEnding', 'YoY_reportedEPS_%', 'reportedEPS']].rename(
        columns={
            'YoY_reportedEPS_%': f'{symbol}_YoY_reportedEPS_%'
            ,'reportedEPS': f'{symbol}_reportedEPS'
            }
    )
    annaul_df['fiscalDateEnding'] = pd.to_datetime(annaul_df['fiscalDateEnding']).dt.year


    if YoY_EPS_screener.empty:
        YoY_EPS_screener = annaul_df
    else:
        YoY_EPS_screener = pd.merge(
            YoY_EPS_screener
            ,annaul_df
            ,on='fiscalDateEnding'
            ,how='outer'
        )


In [89]:
YoY_EPS_screener

,fiscalDateEnding,KR_YoY_reportedEPS_%,KR_reportedEPS
14,2010,NaN,0.0457
13,2011,1799.781182,0.8682
12,2012,-45.081778,0.4768
11,2013,192.093121,1.3927
10,2014,4.200474,1.4512
9,2015,54.575524,2.2432
8,2016,-2.139800,2.1952
7,2017,-7.347850,2.0339
6,2018,0.299916,2.0400
5,2019,2.941176,2.1000


In [95]:
stock_PE_annual_consolidate_df

,fiscalDateEnding,KR_PE
0,2024,12.85
1,2023,10.83
2,2022,12.11
3,2021,13.04
4,2020,14.44
5,2019,13.80
6,2018,13.48
7,2017,13.50
8,2016,15.72
9,2015,18.65


# SP500 sectors

In [81]:
# Function to get the list of S&P 500 companies and their sectors
def get_sp500_companies():
    # Fetch the S&P 500 company symbols and sectors from a reliable source (e.g., Wikipedia)
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    tables = pd.read_html(url)
    
    # Extract the relevant table containing the company symbols and sectors
    sp500_df = tables[0]
    
    # Return the DataFrame containing S&P 500 companies and sectors
    return sp500_df[['Symbol', 'GICS Sector', 'GICS Sub-Industry']]


# Function to create a dictionary of sectors and sub-sectors
def create_sector_subsector_dict(df):
    sector_subsector_dict = {}
    for index, row in df.iterrows():
        sector = row['GICS Sector']
        subsector = row['GICS Sub-Industry']
        if sector not in sector_subsector_dict:
            sector_subsector_dict[sector] = [subsector]
        else:
            sector_subsector_dict[sector].append(subsector)
    return sector_subsector_dict

# Function to filter the S&P 500 companies by sector
def company_sector_list(df, sector):
    return df[df['GICS Sector'] == sector]['Symbol'].tolist()

def company_sub_sector_list(df, sub_sector):
    return df[df['GICS Sub-Industry'] == sub_sector]['Symbol'].tolist()


# Get the list of S&P 500 companies and their sectors
sp500_df  = get_sp500_companies()

sp500_companies_sectors = sp500_df ['GICS Sector'].value_counts().index
sp500_companies_sub_sectors = sp500_df ['GICS Sub-Industry'].value_counts().index

sector_subsector_dict = create_sector_subsector_dict(sp500_df)


# Function to create a DataFrame from the sector_subsector_dict
def create_sector_dataframe():
    # Create a list to store dictionacompany_sector_listries representing each row of data
    data = []
    
    # Filter the DataFrame to get stocks in the specified sector
    for sector in sp500_companies_sectors:
        sector_stocks_list = company_sector_list(sp500_df, sector)

        # Iterate over the stocks in the sector and create a dictionary for each
        for i, ticker in enumerate(sector_stocks_list, start=1):
            # Create a dictionary for the current stock in the sector
            row_data = {'Sector': sector, 'Ticker': ticker}
            # Append the dictionary to the list
            data.append(row_data)
    
    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df


sector_subsector_dict = create_sector_subsector_dict(sp500_df)


sector_ticker_df = create_sector_dataframe()
sector_ticker_list = sector_ticker_df[sector_ticker_df['Sector'] == 'Consumer Staples']['Ticker'].tolist()

In [82]:
# Pivot the DataFrame
pivot_sector_ticker_df = sector_ticker_df.groupby('Sector')['Ticker'].apply(list).reset_index()

# Transpose to get sectors as columns
pivot_sector_ticker_df = pivot_sector_ticker_df.set_index('Sector').T

pivot_sector_ticker_df

Sector,Communication Services,Consumer Discretionary,Consumer Staples,Energy,Financials,Health Care,Industrials,Information Technology,Materials,Real Estate,Utilities
Ticker,"[GOOGL, GOOG, T, CHTR, CMCSA, EA, FOXA, FOX, I...","[ABNB, AMZN, APTV, AZO, BBWI, BBY, BKNG, BWA, ...","[MO, ADM, BF.B, BG, CPB, CHD, CLX, KO, CL, CAG...","[APA, BKR, CVX, COP, CTRA, DVN, FANG, EOG, EQT...","[AFL, ALL, AXP, AIG, AMP, AON, ACGL, AJG, AIZ,...","[ABT, ABBV, A, ALGN, AMGN, BAX, BDX, BIO, TECH...","[MMM, AOS, ALLE, AAL, AME, ADP, AXON, BA, BR, ...","[ACN, ADBE, AMD, AKAM, APH, ADI, ANSS, AAPL, A...","[APD, ALB, AMCR, AVY, BALL, CE, CF, CTVA, DOW,...","[ARE, AMT, AVB, BXP, CPT, CBRE, CSGP, CCI, DLR...","[AES, LNT, AEE, AEP, AWK, ATO, CNP, CMS, ED, C..."


In [83]:
pivot_sector_ticker_df['Consumer Discretionary'].iloc[0]

['ABNB',
 'AMZN',
 'APTV',
 'AZO',
 'BBWI',
 'BBY',
 'BKNG',
 'BWA',
 'CZR',
 'KMX',
 'CCL',
 'CMG',
 'DRI',
 'DECK',
 'DPZ',
 'DHI',
 'EBAY',
 'ETSY',
 'EXPE',
 'F',
 'GRMN',
 'GM',
 'GPC',
 'HAS',
 'HLT',
 'HD',
 'LVS',
 'LEN',
 'LKQ',
 'LOW',
 'LULU',
 'MAR',
 'MCD',
 'MGM',
 'MHK',
 'NKE',
 'NCLH',
 'NVR',
 'ORLY',
 'POOL',
 'PHM',
 'RL',
 'ROST',
 'RCL',
 'SBUX',
 'TPR',
 'TSLA',
 'TJX',
 'TSCO',
 'ULTA',
 'WYNN',
 'YUM']

In [16]:
len(pivot_sector_ticker_df['Financials'].iloc[0])

71

## Appendix

In [131]:
# Consumer Staples

ticker_symbols = [
    'MO',
    'ADM',
    # 'BF.B', # DATA ISSUE
    'BG',
    'CPB',
    'CHD',
    'CLX',
    'KO',
    'CL',
    'CAG',
    'STZ', # !!
    'COST',
    'DG',
    'DLTR', # !!
    'EL',
    'GIS',
    'HSY',
    'HRL',
    'K',
    'KVUE',
    'KDP',
    'KMB',
    'KHC',
    # 'KR', # DATA ISSUE
    'LW',
    # 'MKC', # DATA ISSUE
    'TAP',
    'MDLZ',
    'MNST',
    'PEP',
    'PM',
    'PG',
    'SJM',
    'SYY',
    'TGT', # !!
    'TSN', # !!
    # 'WBA', # DATA ISSUE
    'WMT'
]

In [132]:
# Energy

ticker_symbols = [
'APA',
 'BKR', # !! HIGH PE, LOW PROFITABILITY, HIGH FORECAST,  BUT CANNOT BEAT FORECAST, 总结与TRGP类似, 基本面更稳妥, 盈利预期高, 有待考证
 'CVX',
 'COP',
 'CTRA',
 'DVN',
 'FANG',
 'EOG',
 'EQT',
 'XOM',
 'HAL', # !!!! 与SLB类似
 'HES', # !  HIGH PE, UNDERVALUE, 市场炒作高预期, 2024表现强劲, 已经被超热了
 'KMI',
 'MRO',
 'MPC',
 'OXY',
 'OKE', # !!
 'PSX',
 'SLB', # !!! A BIT HIGH PE, GOOD ROE, MED PROFITABILITY, STEADY GROWTH FORECAST, AND CAN BEAT FORECAST, 总结基本面良好，与HAL类型相似, 市场炒作热度比HAL更高, 值得研究
 'TRGP', # !！ HGIH PE, HIGH ROE, RELATIVE MORE DEBT THAN EQUITY, LOW PROFITABILITY, HIGH FORECAST, BUT CANNOT BEAT FORECAST, 总结基本面一般, 但是盈利预期非常高, 有待考证
 'VLO',
 'WMB'
 ]

In [135]:
electric_utility_symbols = [
'CEPU'
,'OTTR'
,'PAM'
,'SO' ##
,'DUK'
,'NRG'
,'IDA'
,'DTE' #
,'CEG' ##
,'SRE'
,'ETR'
,'PEG'
,'MGEE'
,'LNT'
,'BKH'
]

In [41]:
# for i in earning_calendar: comment out the for loop in case of future usage, i can be the parameter of {}month
CSV_URL = f'https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol=GOOG&horizon=12month&apikey={alpha_vantage_api_key}'
with requests.Session() as s:
    download = s.get(CSV_URL)
    decoded_content = download.content.decode('utf-8')
    cr = csv.reader(decoded_content.splitlines(), delimiter=',')
    my_list = list(cr)

In [42]:
my_list

[['symbol', 'name', 'reportDate', 'fiscalDateEnding', 'estimate', 'currency'],
 ['GOOG', 'Alphabet Inc - Class C', '2025-01-28', '2024-12-31', '', 'USD'],
 ['GOOG', 'Alphabet Inc - Class C', '2025-04-23', '2025-03-31', '', 'USD'],
 ['GOOG', 'Alphabet Inc - Class C', '2025-07-21', '2025-06-30', '', 'USD']]